In [1]:
# the purpose of this notebook is to finally get to using random forest classification (and maybe other
# machine learning tools depending on what is found) to see if the altitude of a consumer drone can be determined
# by nothing but its struggle to stay upright against different wind (near-ground, in shear-zone, and laminar)

In [2]:
# import tings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import plot_tree

In [3]:
df = pd.read_csv('windowed_data.csv')
test_df = pd.read_csv('FlightLogs/Jan13.5-38.csv')
train_df = df[~df['file_name'].str.contains('Jan13.5')] # we're gonna test the train_df on the Jan13.5 logs now

In [4]:
X_train = train_df.loc[:, 'speed_avg':'turbulence_intensity']
Y_train = train_df['altitude(m)']

In [5]:
# splitting X and Y into testing and training, unnecessary for now since i manually separated train_df from the Jan13.5 logs I'll test them on
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=67, test_size=0.25, stratify=Y)
#X_test = test_df.loc[:, 'speed_avg':'turbulence_intensity']

In [6]:

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
#Y_hat = rf.predict(X_test)

RandomForestClassifier()

In [7]:
#rf.score(X_test, Y_test)

NameError: name 'X_test' is not defined

In [8]:
# confusion matrix to determine where the model is going wrong vs right

Y_test_str = Y_test.astype(str)
Y_hat_str = Y_hat.astype(str)
cm = confusion_matrix(Y_test_str, Y_hat_str, labels=['10', '30', '50'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['10m (Ground)', '30m (Shear)', '50m (Laminar)'],
            yticklabels=['10m (Ground)', '30m (Shear)', '50m (Laminar)'])
plt.xlabel('Model Prediction')
plt.ylabel('Actual Altitude')
plt.title('Confusion Matrix: The Blind Altimeter')
plt.show()

NameError: name 'Y_test' is not defined

In [9]:
# plot tree, cool to see but no real analysis to collect from it

plt.figure(figsize=(20,10))
plot_tree(rf.estimators_[0], 
          feature_names=X_train.columns, 
          filled=True, 
          rounded=True, 
          max_depth=2) # Limiting depth for readability
plt.show()

In [ ]:
features = pd.DataFrame(rf.feature_importances_, index = X.columns).sort_values(by = 0, ascending=False)


✅ Accuracy on Stable Auto-Flight Sections: 67.5%


In [58]:
# this cell handles the validating part of the model. feed it a random file that the model wasn't trained on and see how it performs.
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

TESTING_FILE_PATH = "RawFlightLogs/Raw_Jan13.5-38.csv"

# ========= BEGIN FUNCTION ============

def validate_model_on_log(filepath, model):

    # read the base file
    flight_df = pd.read_csv(filepath)
    flight_df.columns = flight_df.columns.str.lstrip()

    # making the new columns the raw file doesn't have
    flight_df['tilt_magnitude'] = np.sqrt(flight_df['pitch(degrees)']**2 + flight_df['roll(degrees)']**2)
    flight_df['power(watt)'] = flight_df['voltage(v)'] * flight_df['current(A)']
    flight_df['timestamp'] = flight_df['datetime(utc)']

    # find height
    if 'height_above_takeoff(meters)' in flight_df.columns:
        pass # we good
    elif 'height_above_takeoff(feet)' in flight_df.columns:
        flight_df['height_above_takeoff(meters)'] = flight_df['height_above_takeoff(feet)'] * 0.3048
    else:
        raise KeyError("There's no altitude column. Check your file once more.")

    # ============== sliding window logic is mostly borrowed from AggregateLogs.ipynb ==============

    WINDOW_SIZE = 50
    STEP_SIZE = 5
    windows = []
    
    for i in range(0, len(flight_df) - WINDOW_SIZE, STEP_SIZE):
        window = flight_df.iloc[i:i + WINDOW_SIZE]
    
        # check that user isn't moving (throttling the drone)
        # if there is any throttling in the window, assume it is NOT auto.
        # will have to do something about the drone continuing to move for a second after releasing the throttle... confounds the data a bit
        if 'rc_throttle' in window.columns:
            is_auto = (window['rc_throttle'] == 1024).all() 
        else:
            flight_df = flight_df.copy()
            print("No rc_throttle column found. Assuming full file is auto-flight.")
        
        altitude_avg = window['height_above_takeoff(meters)'].mean()
        
        # track vertical climb or drop speed to see when drone is moving up or down
        zSpeed_avg = window['zSpeed(mph)'].abs().mean()
        is_stable = zSpeed_avg < 0.5

        # only if both of these are true is it valid for prediction
        valid_for_prediction = is_auto and is_stable
        
        # define correct answer but ignore transitory zones
        true_zone = -1
        if 7 <= altitude_avg <= 13:
            true_zone = 10
        elif 27 <= altitude_avg <= 33:
            true_zone = 30
        elif 47 <= altitude_avg <= 53:
            true_zone = 50

        # calculate the features (for model input)
        stats = {
            'timestamp': window['timestamp'].iloc[-1],
            'true_altitude_m': altitude_avg,
            'true_zone': true_zone,
            'is_stable': is_stable,
            'valid_for_prediction': valid_for_prediction,
            # the 8 Features the model needs:
            'speed_avg': window['speed(mph)'].mean(),
            'speed_sd': window['speed(mph)'].std(),
            'power_avg': window['power(watt)'].mean(),
            'power_sd': window['power(watt)'].std(),
            'power_intensity': window['power(watt)'].std() / (window['power(watt)'].mean() + 0.05),
            'tilt_avg': window['tilt_magnitude'].mean(),
            'tilt_sd': window['tilt_magnitude'].std(),
            'turbulence_intensity': window['tilt_magnitude'].std() / (window['tilt_magnitude'].mean() + 0.05)
        }
        windows.append(stats)
        
    validation_df = pd.DataFrame(windows)

    # Finally, get all the feature columns from validation_df and do the final predicition on the (previously) raw untrained log
    feature_cols = ['speed_avg', 'speed_sd', 'power_avg', 'power_sd', 
                    'power_intensity', 'tilt_avg', 'tilt_sd', 'turbulence_intensity']

    # predictions without accounting for drone manually climbing or falling
    raw_predictions = model.predict(validation_df[feature_cols]) 
    
    # If stable -> Use Model Prediction
    # If unstable -> Set to 0 (or a special code for "Transition")
    validation_df['predicted_zone'] = np.where(validation_df['valid_for_prediction'], raw_predictions, np.nan)
    
    return validation_df

# ============== END FUNCTION ================
# now we run the function and do the real test

val_df = validate_model_on_log(TESTING_FILE_PATH, rf)

valid_windows = val_df.dropna(subset=['predicted_zone'])
valid_windows = valid_windows[valid_windows['true_zone'] != -1] # exists solely to calculate accuracy_score. must be valid for prediction AND be in the 10, 30, 50m range

# getting accuracy and checking that a valid window exists
if not valid_windows.empty:
    acc = accuracy_score(valid_windows['true_zone'], valid_windows['predicted_zone'])
    print(f"\n✅ Accuracy on Stable Auto-Flight Sections: {acc:.1%}")
else:
    print("\n⚠️ No stable auto-flight sections found for accuracy check.")

# ========= PLOT THE RESULT =============
plt.figure(figsize=(14, 6))

# Convert timestamps to elapsed seconds
times = pd.to_datetime(val_df['timestamp'])
elapsed = (times - times.iloc[0]).dt.total_seconds() # new column of seconds elapsed since start
val_df['elapsed_time'] = elapsed # save to df for easy grouping

# plot line that follows altitude throughout
plt.plot(val_df['elapsed_time'], val_df['true_altitude_m'], 
         color='black', alpha=0.3, linewidth=2, label='True Altitude')

# filter for valid predictions
scatter_data = val_df.dropna(subset=['predicted_zone']) # exists for the graph. only needs the data to be valid for prediction (in case drone is at 20m, it woul

# plot predictions (the model)
# Green=10m, Orange=30m, Blue=50m
color_map = {10: 'green', 30: 'orange', 50: 'blue'}
pred_colors = [color_map.get(p, 'gray') for p in scatter_data['predicted_zone']]

plt.scatter(scatter_data['elapsed_time'], scatter_data['true_altitude_m'], 
            c=pred_colors, s=30, alpha=0.8, label='Model Prediction')

# titles, labels, all that jazz
plt.title(f"Blind Altimeter Validation: {''.join(TESTING_FILE_PATH.split('/')[1:])}", fontsize=14)
plt.xlabel("Time (Seconds)")
plt.ylabel("Altitude (m)")
plt.grid(True, linestyle='--', alpha=0.5)

# custom Legend
patches = [
    mpatches.Patch(color='green', label='Pred: 10m'),
    mpatches.Patch(color='orange', label='Pred: 30m'),
    mpatches.Patch(color='blue', label='Pred: 50m'),
    Line2D([0], [0], color='black', alpha=0.3, lw=2, label='Actual Altitude')
]
plt.legend(handles=patches, loc='best')

plt.show()